<a href="https://colab.research.google.com/github/Jessietbl/aviation-scsirisk-showcase/blob/main/04_mr_tft_bhm_risk_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MR-TFT Showcase (Lightweight, Dummy Data)

This mini notebook uses **dummy monthly SCSI + revenue** and a tiny baseline model
to demonstrate the pipeline without GPU or heavy installs.

Files expected in `/content/data` (already included in the repo ZIP):
- `final_data_with_scsi.csv` (Date, SCSI)
- `monthly_revenue_piecewise_pchip.csv` (Date, revenue_RM_monthly)
- `monthly_posteriors.csv` (Date, p_revenue)

In [ ]:
# Cell 0 — helper module (lightweight; no OCR/GPUs required)
mkdir -p src
cat > src/mrtft_bhm.py <<'PY'
# -*- coding: utf-8 -*-
"""
mrtft_bhm.py — tiny utilities for an MR-TFT + BHM showcase.

What’s inside (light + robust):
- Monthly I/O helpers (date detection, monthly grid, safe merges)
- Posterior handling (attach/engineer p_revenue)
- Fourier + multi-resolution features (lags/rolling stats)
- Minimal TFT dataset scaffolding (optional)
- Simple metrics + Bayesian VaR

This demo stays light: no OCR; if you want OCR+BHM later, you can extend.
"""
from __future__ import annotations
import os, re
from pathlib import Path
from typing import Optional, Tuple, List, Dict
import numpy as np
import pandas as pd

# ---------------------------
# Monthly I/O helpers
# ---------------------------
def _best_date_column(df: pd.DataFrame) -> str:
    for k in ("Date","date","month_end","month","Month","period","timestamp","Unnamed: 0","unnamed: 0"):
        if k in df.columns: return k
    for c in df.columns:
        s = pd.to_datetime(df[c], errors="coerce")
        if s.notna().mean() > 0.6: return c
    raise ValueError("No date-like column found.")

def _coerce_to_month(x) -> pd.Series:
    s = pd.to_datetime(x, errors="coerce")
    return s.dt.to_period("M").dt.to_timestamp()

def ensure_monthly(df: pd.DataFrame) -> pd.DataFrame:
    if "Date" not in df.columns:
        raise KeyError("ensure_monthly expects a 'Date' column.")
    d = df.copy()
    d["Date"] = _coerce_to_month(d["Date"])
    d = d.dropna(subset=["Date"]).sort_values("Date")
    if d["Date"].duplicated().any():
        num = d.select_dtypes(include=[np.number]).columns.tolist()
        non = [c for c in d.columns if c not in set(num + ["Date"])]
        agg = {**{c: "mean" for c in num}, **{c: "first" for c in non}}
        d = d.groupby("Date", as_index=False).agg(agg)
    if d.empty: return d
    idx = pd.date_range(d["Date"].min(), d["Date"].max(), freq="MS")
    d = d.set_index("Date").reindex(idx)
    d.index.name = "Date"
    return d.reset_index()

def merge_monthly_left(left: pd.DataFrame, right: pd.DataFrame | None) -> pd.DataFrame:
    L = left.copy()
    if "Date" not in L.columns:
        L = L.rename(columns={_best_date_column(L): "Date"})
    L = ensure_monthly(L)
    if right is None or right.empty:
        return L
    R = right.copy()
    if "Date" not in R.columns:
        R = R.rename(columns={_best_date_column(R): "Date"})
    R = ensure_monthly(R)
    return L.merge(R, on="Date", how="left")

def detect_columns(df: pd.DataFrame,
                   scsi_candidates=("SCSI","scsi","y","target","index","value"),
                   rev_candidates=("revenue","revenue_rm_monthly","rev","monthly_revenue")) -> Dict[str,str|None]:
    low = {c.lower(): c for c in df.columns}
    def pick(cands):
        for n in cands:
            if n.lower() in low: return low[n.lower()]
        return None
    return {"scsi": pick(scsi_candidates), "revenue": pick(rev_candidates)}

def load_unified_monthly(path_scsi: str,
                         path_revenue: Optional[str] = None,
                         scsi_name: Optional[str] = None) -> pd.DataFrame:
    scsi_raw = pd.read_csv(path_scsi)
    scsi_raw = scsi_raw.rename(columns={_best_date_column(scsi_raw): "Date"})
    scsi_raw["Date"] = _coerce_to_month(scsi_raw["Date"])
    scsi_col = scsi_name or detect_columns(scsi_raw).get("scsi") or "SCSI"
    scsi_raw[scsi_col] = pd.to_numeric(scsi_raw[scsi_col], errors="coerce")
    scsi = ensure_monthly(scsi_raw[["Date", scsi_col]].rename(columns={scsi_col: "SCSI"}))

    revenue = pd.DataFrame(columns=["Date","revenue"])
    if path_revenue and os.path.exists(path_revenue):
        rev_raw = pd.read_csv(path_revenue)
        rev_raw = rev_raw.rename(columns={_best_date_column(rev_raw): "Date"})
        rev_raw["Date"] = _coerce_to_month(rev_raw["Date"])
        rev_col = next((c for c in rev_raw.columns if c!="Date" and
                        pd.to_numeric(rev_raw[c], errors="coerce").notna().mean()>0.6), None)
        if rev_col:
            revenue = ensure_monthly(rev_raw[["Date", rev_col]].rename(columns={rev_col: "revenue"}))

    d = merge_monthly_left(scsi, revenue)
    for c in ("SCSI","revenue"):
        if c in d.columns:
            s = pd.to_numeric(d[c], errors="coerce")
            try: d[c] = s.interpolate("time").ffill().bfill()
            except: d[c] = s.interpolate().ffill().bfill()
    return d

# ---------------------------
# Posterior features (p_revenue)
# ---------------------------
def _ensure_posterior_engineered(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    if "p_revenue" not in out.columns: out["p_revenue"] = 0.33
    out["p_revenue"] = pd.to_numeric(out["p_revenue"], errors="coerce").clip(0.05, 0.95)
    out["p_revenue"] = out["p_revenue"].ewm(alpha=0.5, adjust=False).mean()
    for L in (1,3,6):
        c = f"p_revenue_lag{L}"
        if c not in out.columns: out[c] = out["p_revenue"].shift(L)
    for W in (3,6):
        c = f"p_revenue_rm{W}"
        if c not in out.columns: out[c] = out["p_revenue"].rolling(W, min_periods=1).mean()
    eng = [c for c in out.columns if c.startswith("p_revenue_")]
    if eng: out[eng] = out[eng].ffill().bfill()
    return out

def attach_posterior_safe(df: pd.DataFrame, monthly_csv: Optional[str] = None,
                          default: float = 0.33, time_col: str = "Date",
                          scsi_col: Optional[str] = None) -> pd.DataFrame:
    out = df.copy()
    if time_col not in out.columns:
        out = out.rename(columns={_best_date_column(out): time_col})
    out[time_col] = _coerce_to_month(out[time_col])
    if "p_revenue" not in out.columns: out["p_revenue"] = np.nan
    if monthly_csv and os.path.exists(monthly_csv):
        p = pd.read_csv(monthly_csv)
        if "Date" not in p.columns:
            for c in p.columns:
                if "date" in c.lower() or "month" in c.lower(): p = p.rename(columns={c:"Date"}); break
        p["Date"] = _coerce_to_month(p["Date"])
        prob = next((k for k in ("p_revenue","posterior","p_rev","P_Revenue") if k in p.columns), None)
        if prob:
            p = p[["Date", prob]].rename(columns={prob:"p_revenue"})
            p["p_revenue"] = pd.to_numeric(p["p_revenue"], errors="coerce")
            p = p.dropna(subset=["Date"]).groupby("Date", as_index=False)["p_revenue"].mean()
            out = out.merge(p, left_on=time_col, right_on="Date", how="left", suffixes=("", "_post"))
            out["p_revenue"] = out["p_revenue"].combine_first(out["p_revenue_post"])
            out.drop(columns=[c for c in ("Date","p_revenue_post") if c in out.columns], inplace=True, errors="ignore")
    out["p_revenue"] = pd.to_numeric(out["p_revenue"], errors="coerce").fillna(default).clip(0,1)
    return _ensure_posterior_engineered(out)

# ---------------------------
# Fourier + multi-resolution features
# ---------------------------
def add_fourier_terms(df: pd.DataFrame, time_col="Date", K=3, period=12) -> pd.DataFrame:
    d = df.copy()
    d[time_col] = pd.to_datetime(d[time_col], errors="coerce")
    d = d.dropna(subset=[time_col]).sort_values(time_col).reset_index(drop=True)
    m_idx = (d[time_col].dt.year*12 + d[time_col].dt.month) - (d[time_col].dt.year.min()*12 + d[time_col].dt.month.min())
    m_idx = m_idx.astype(float)
    for k in range(1, K+1):
        ang = 2*np.pi*k*(m_idx/float(period))
        d[f"m_sin_{k}"] = np.sin(ang); d[f"m_cos_{k}"] = np.cos(ang)
    return d

def create_multi_resolution_features(df: pd.DataFrame, time_col="Date", target="SCSI",
                                     mr_windows=(3,6,12), lag_features=(1,3,6,12)) -> Tuple[pd.DataFrame, List[str]]:
    d = df.copy()
    d[time_col] = pd.to_datetime(d[time_col], errors="coerce")
    d = d.dropna(subset=[time_col]).sort_values(time_col).reset_index(drop=True).set_index(time_col)
    y = pd.to_numeric(d[target], errors="coerce")
    # yoy
    d[f"{target}_yoy"] = y - y.shift(12)
    d[f"{target}_yoy_abs"] = (y - y.shift(12)).abs()
    mr = sorted(set(list(mr_windows)+[3,6,12])); lags = sorted(set(list(lag_features)+[1,3,6,12]))
    for w in mr:
        d[f"{target}_ma_{w}"] = y.rolling(w, min_periods=1).mean()
        d[f"{target}_std_{w}"] = y.rolling(w, min_periods=1).std().fillna(0.0)
        d[f"{target}_range_{w}"] = y.rolling(w, min_periods=1).max() - y.rolling(w, min_periods=1).min()
        d[f"{target}_ewm_{w}"] = y.ewm(span=w, adjust=False, min_periods=1).mean()
    for L in lags:
        d[f"{target}_lag_{L}"] = y.shift(L)
        d[f"{target}_diff_{L}"] = y.diff(L)
        pc = y.pct_change(L); d[f"{target}_pct_change_{L}"] = pc.replace([np.inf,-np.inf], np.nan)
        if L<=6: d[f"{target}_lag_{L}_ma3"] = d[f"{target}_lag_{L}"].rolling(3, min_periods=1).mean()
    d["month"] = d.index.month; d["quarter"] = d.index.quarter
    d["month_sin"] = np.sin(2*np.pi*d["month"]/12.0); d["month_cos"] = np.cos(2*np.pi*d["month"]/12.0)
    d["quarter_sin"] = np.sin(2*np.pi*d["quarter"]/4.0); d["quarter_cos"] = np.cos(2*np.pi*d["quarter"]/4.0)
    d["time_trend"] = np.arange(len(d), dtype=int)
    d = d.reset_index()
    num = d.select_dtypes(include=[np.number]).columns
    d[num] = d[num].replace([np.inf,-np.inf], np.nan).ffill().bfill().fillna(0.0)
    unknown_allow = (
        [f"{target}_lag_{L}" for L in (1,3,6,12)] +
        [f"{target}_ma_{w}" for w in (3,6,12)] +
        [f"{target}_ewm_{w}" for w in (3,6,12)] +
        [f"{target}_diff_{L}" for L in (1,3,6,12)] +
        [f"{target}_pct_change_{L}" for L in (1,3,6,12)] +
        [f"{target}_yoy", f"{target}_yoy_abs"]
    )
    unknown_allow = [c for c in unknown_allow if c in d.columns]
    return d, unknown_allow

# ---------------------------
# Simple metrics + Bayesian VaR
# ---------------------------
def eval_rmse_mae_r2(y_true, y_pred):
    y_true = np.asarray(y_true, float).reshape(-1)
    y_pred = np.asarray(y_pred, float).reshape(-1)
    rmse = float(np.sqrt(np.mean((y_true - y_pred)**2)))
    mae  = float(np.mean(np.abs(y_true - y_pred)))
    denom = np.sum((y_true - np.nanmean(y_true))**2)
    r2 = float("nan") if denom<=1e-12 else 1.0 - float(np.nansum((y_true - y_pred)**2))/denom
    return rmse, mae, r2

def _normalize_posterior_df(posterior_df: pd.DataFrame) -> pd.DataFrame:
    p = posterior_df.copy()
    if "Date" not in p.columns:
        for c in p.columns:
            if "date" in c.lower() or "month" in c.lower(): p = p.rename(columns={c:"Date"}); break
    p["Date"] = _coerce_to_month(p["Date"])
    pv = next((k for k in ("p_revenue","posterior","pRev","p_rev") if k in p.columns), None)
    if pv is None: raise KeyError("Posterior frame missing probability column.")
    p[pv] = pd.to_numeric(p[pv], errors="coerce")
    return p.dropna(subset=["Date"]).groupby("Date", as_index=False)[pv].mean().rename(columns={pv:"p_revenue"})

def _ensure_p_revenue_col(preds_df, posterior_df, time_col="Date", fallback=0.33):
    preds = preds_df.copy(); preds[time_col] = _coerce_to_month(preds[time_col])
    post = _normalize_posterior_df(posterior_df)
    m = preds.merge(post, left_on=time_col, right_on="Date", how="left", suffixes=("", "_post"))
    if "Date" in m.columns and time_col!="Date": m.drop(columns=["Date"], inplace=True, errors="ignore")
    if "p_revenue" not in m.columns: m["p_revenue"] = np.nan
    if "p_revenue_post" in m.columns:
        m["p_revenue"] = m["p_revenue"].combine_first(m["p_revenue_post"]); m.drop(columns=["p_revenue_post"], inplace=True)
    m["p_revenue"] = pd.to_numeric(m["p_revenue"], errors="coerce").ffill().bfill().fillna(float(fallback)).clip(1e-6,1-1e-6)
    return m

def calculate_bayesian_var(posterior_df: pd.DataFrame, predictions_df: pd.DataFrame,
                           kappa: float = 100.0, flat_revenue: float = 1_000_000.0,
                           full_df: Optional[pd.DataFrame] = None) -> pd.DataFrame:
    from scipy.stats import beta
    preds = predictions_df.copy(); preds["Date"] = _coerce_to_month(preds["Date"])
    if "y_pred" not in preds.columns:
        for c in ("pred","yhat","y_pred_cal","yhat_smoothed"):
            if c in preds.columns: preds = preds.rename(columns={c:"y_pred"}); break
    if "y_pred" not in preds.columns: raise KeyError("Predictions need a 'y_pred' column (or alias).")
    merged = _ensure_p_revenue_col(preds, posterior_df, time_col="Date", fallback=0.33)
    if full_df is not None and "revenue" in full_df.columns:
        r = full_df[["Date","revenue"]].copy(); r["Date"] = _coerce_to_month(r["Date"])
        merged = merged.merge(r, on="Date", how="left")
    rows = []
    for _, r in merged.iterrows():
        p = float(r["p_revenue"]); a=max(p*kappa,1e-6); b=max((1-p)*kappa,1e-6)
        rev = float(r["revenue"]) if "revenue" in r and pd.notna(r["revenue"]) else flat_revenue
        scsi_factor = 1.0 + float(r["y_pred"])/10.0
        for q in (0.05,0.50,0.95):
            val = float(beta.ppf(q,a,b))*rev*scsi_factor
            rows.append({"Date": r["Date"], f"VaR_{int(q*100)}": val})
    out = pd.DataFrame(rows).groupby("Date", as_index=False).mean().sort_values("Date")
    return out

# ---------------------------
# (Optional) minimal TFT dataloaders for later
# ---------------------------
def build_datasets_with_tail(train_df, val_df, test_df, *,
                             target="SCSI", time_col="Date", group_col="scsi_series",
                             enc_len=12, pred_len=1, add_target_scales=True, allow_missing_timesteps=True, normalizer=None):
    from pytorch_forecasting import TimeSeriesDataSet
    # train
    tr = train_df.copy()
    if group_col not in tr.columns: tr[group_col] = "scsi_series"
    tr[time_col] = _coerce_to_month(tr[time_col])
    tr[target] = pd.to_numeric(tr[target], errors="coerce")
    tr["time_idx"] = (tr.groupby(group_col)[time_col].rank(method="first").astype(int) - 1)
    from pytorch_forecasting.data import GroupNormalizer
    norm = normalizer or GroupNormalizer(groups=[group_col])
    ds_tr = TimeSeriesDataSet(
        tr, time_idx="time_idx", target=target, group_ids=[group_col],
        min_encoder_length=max(1, min(6, enc_len)), max_encoder_length=int(enc_len),
        min_prediction_length=int(pred_len), max_prediction_length=int(pred_len),
        time_varying_known_reals=[c for c in ["month_sin","month_cos","quarter_sin","quarter_cos","time_trend","p_revenue"] if c in tr.columns],
        time_varying_unknown_reals=[target],
        static_categoricals=[group_col], target_normalizer=norm,
        allow_missing_timesteps=allow_missing_timesteps, add_relative_time_idx=True,
        add_target_scales=bool(add_target_scales), add_encoder_length=True,
    )
    tail = train_df.sort_values(time_col).tail(min(enc_len, len(train_df)))
    ds_va = TimeSeriesDataSet.from_dataset(ds_tr, pd.concat([tail, val_df], ignore_index=True), stop_randomization=True)
    ds_te = TimeSeriesDataSet.from_dataset(ds_tr, pd.concat([tail, test_df], ignore_index=True), stop_randomization=True)
    return ds_tr, ds_va, ds_te

def make_dataloaders(ds_train, ds_val=None, ds_test=None, *, batch_size=1, num_workers=0, shuffle_train=True, pin_memory=True):
    def _dl(ds, train):
        if ds is None: return None
        return ds.to_dataloader(train=train, batch_size=int(batch_size), num_workers=int(num_workers),
                                pin_memory=pin_memory, shuffle=(shuffle_train if train else False))
    return _dl(ds_train, True), _dl(ds_val, False), _dl(ds_test, False)

def tft_predict_last_horizon(model, dataset, batch_size=1) -> np.ndarray:
    dl = dataset.to_dataloader(train=False, batch_size=int(batch_size), num_workers=0, pin_memory=True)
    raw = model.predict(dl, mode="prediction", return_x=False) if hasattr(model,"predict") else model.predict(dl)
    arr = np.asarray(raw)
    if arr.ndim == 3: return np.median(arr[:,-1,:], axis=1)
    if arr.ndim == 2: return arr[:,-1]
    return arr.reshape(-1)

PY


In [ ]:
# Cell 1 — imports & paths
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from src.mrtft_bhm import (
    load_unified_monthly, attach_posterior_safe, add_fourier_terms, create_multi_resolution_features,
    eval_rmse_mae_r2, calculate_bayesian_var
)

DATA_DIR = Path("/content/data")
OUT_DIR  = Path("/content/out"); OUT_DIR.mkdir(parents=True, exist_ok=True)

SCSI_CSV   = DATA_DIR / "final_data_with_scsi.csv"
REV_CSV    = DATA_DIR / "monthly_revenue_piecewise_pchip.csv"
POST_CSV   = DATA_DIR / "monthly_posteriors.csv"

# Cell 2 — make dummy data if files not present
DATA_DIR.mkdir(parents=True, exist_ok=True)
if not SCSI_CSV.exists() or not REV_CSV.exists() or not POST_CSV.exists():
    dates = pd.date_range("2019-01-01","2024-12-01",freq="MS")
    m = np.arange(len(dates))
    rng = np.random.default_rng(7)

    scsi = pd.DataFrame({
        "Date": dates,
        "SCSI": 50 + 10*np.sin(2*np.pi*m/12) + 3*np.sin(2*np.pi*m/6) + rng.normal(0,1.2,len(m))
    })
    scsi.to_csv(SCSI_CSV, index=False)

    revenue = pd.DataFrame({
        "Date": dates,
        "revenue_RM_monthly": 8e6 + 2e6*np.sin(2*np.pi*(m-2)/12) + rng.normal(0,2.5e5,len(m))
    })
    revenue.to_csv(REV_CSV, index=False)

    # seasonal posterior (Q4 > Q1 > Q2 ~= Q3)
    base = np.where((pd.to_datetime(dates).month>=10)|(pd.to_datetime(dates).month<=3), 0.36, 0.30)
    post = pd.DataFrame({"Date": dates, "p_revenue": np.clip(base + rng.normal(0,0.02,len(m)), 0.1, 0.9)})
    post.to_csv(POST_CSV, index=False)

print("Data files ready:", SCSI_CSV.exists(), REV_CSV.exists(), POST_CSV.exists())

# Cell 3 — unify (SCSI+Revenue) and attach posterior
df = load_unified_monthly(str(SCSI_CSV), str(REV_CSV))
df = attach_posterior_safe(df, monthly_csv=str(POST_CSV), default=0.33)

# Add features
df = add_fourier_terms(df, time_col="Date", K=3, period=12)
df_feat, _ = create_multi_resolution_features(df, time_col="Date", target="SCSI")

df_feat.head()


In [ ]:
# Cell 4 — split & seasonal-naive baseline (12-month lag)
train = df_feat[df_feat["Date"].dt.year <= 2022].copy()
test  = df_feat[df_feat["Date"].dt.year == 2023].copy()

def seasonal_naive(train_df, test_df, col="SCSI", season=12):
    tr = train_df[["Date",col]].dropna().set_index("Date")[col]
    preds = []
    for d in test_df["Date"]:
        lag = d - pd.DateOffset(months=season)
        preds.append(float(tr.get(lag, tr.mean())))
    return np.array(preds, float)

y_true = test["SCSI"].to_numpy(float)
y_pred = seasonal_naive(train, test, "SCSI", 12)

rmse, mae, r2 = eval_rmse_mae_r2(y_true, y_pred)
print(f"Baseline metrics  •  RMSE={rmse:.3f}  MAE={mae:.3f}  R2={r2:.3f}")

# Save CSVs
pred_df = test[["Date"]].copy()
pred_df["y_true"] = y_true
pred_df["y_pred"] = y_pred
pred_df.to_csv(OUT_DIR/"showcase_pred_vs_actual.csv", index=False)

pd.DataFrame([{"metric":"RMSE","value":rmse},{"metric":"MAE","value":mae},{"metric":"R2","value":r2}])\
  .to_csv(OUT_DIR/"showcase_metrics.csv", index=False)

print("Saved:",
      OUT_DIR/"showcase_pred_vs_actual.csv",
      OUT_DIR/"showcase_metrics.csv")


In [ ]:
# Cell 5 — quick plot
plt.figure(figsize=(10,4))
plt.plot(df["Date"], df["SCSI"], label="Actual SCSI")
plt.plot(pred_df["Date"], pred_df["y_pred"], "--", label="Baseline (seasonal-naive)")
plt.title("Showcase: Actual vs Baseline Prediction"); plt.grid(True, alpha=.3); plt.legend(); plt.show()


In [ ]:
# Cell 6 — Bayesian VaR (optional)
var_df = calculate_bayesian_var(
    posterior_df = pd.read_csv(POST_CSV),
    predictions_df = pred_df[["Date","y_pred"]],
    kappa = 80.0,            # prior strength
    flat_revenue = 1_000_000,
    full_df = df             # uses df['revenue'] if present
)
var_path = OUT_DIR/"showcase_bayesian_var.csv"
var_df.to_csv(var_path, index=False)
print("Saved:", var_path)
var_df.tail(3)
